# Desafio Final - Dupla Echo

# Introdução

O desafio final consiste na implementação de um modelo classificador inteligente que seja capaz de analisar imagens do bioma pantanal e dessa forma indicar se há ou não alguma das espécies de animais que foram listadas (onça-pintada, lobo guará, ariranha, tatu canastra, tamanduá bandeira, jacaré do papo amarelo, sucuri, tucano, piranha e capivara).

O primeiro passo é montar o drive, dessa forma o *Jupyter Notebook* terá acesso aos conteúdos presentes no Google Drive, possibilitando a execução do *Notebook* pela nuvem.

In [ ]:
# Montando o drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Para a resolução do problema o time optou por usar uma Rede Neural Convolucional. Algumas bibliotecas serão utilizadas e para isso, serão importadas logo abaixo:

In [ ]:
# Utilitários
import sys
import os
import time
#import matplotlib.pyplot as plt
#import numpy as np
#%matplotlib inline
import torch
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch import nn
from torch import optim

Com o intuito de otimizar o processo de treinamento da rede, a célula abaixo testa se está disponível uma GPU, caso contrário será utilizada a CPU.

In [ ]:
def testar_gpu():
	train_on_gpu = torch.cuda.is_available() #Observa se a GPU está disponivel
	if train_on_gpu: #Se sim
		device = torch.device('cuda') #Seleciona o device como GPU
		print("Treinando na GPU") #E manda a mensagem
	else: #Se não
		device = torch.device('cpu') #Seleciona o device como cpu
		print("GPU indisponível, treinando na CPU") #E avisa que a GPU não esta disponível
	return device

device = testar_gpu()

Treinando na GPU


# Criação do Dataset com FastAi

O FastAI é uma biblioteca de Machine Learning que será utilizada no problema com o intuito de facilitar a criação do dataset.

In [ ]:
# Instalamos a bilioteca para a utilização no Notebook
!pip install git+https://github.com/fastai/fastai.git

  Cloning https://github.com/fastai/fastai.git to /tmp/pip-req-build-ssetpgtd
  Running command git clone -q https://github.com/fastai/fastai.git /tmp/pip-req-build-ssetpgtd
     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for fastai: filename=fastai-2.3.1-cp37-none-any.whl size=193488 sha256=3e2a3f105d92c4f382370d1409b1c7f07d18ce42161c66a9819fb65fbffe1584
  Stored in directory: /tmp/pip-ephem-wheel-cache-u8y8b0mg/wheels/cf/46/39/b2d08762125ed2376861976ab2c4ac30c029b86e375735d9b8
Successfully built fastai
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


A importação da biblioteca FastAI é realizada:

In [ ]:
import fastai.vision.all as fst

Definimos as duas variáveis *dataset_path* e *destionation_path* que irão armazenar o diretório com os arquivos csv e as imagens, respectivamente.

In [ ]:

dataset_path = '/content/drive/My Drive/IMLDL/IMLDL-Desafio-DataSet'
destination_path = '/content/drive/My Drive/IMLDL/IMLDL-Desafio-Imagens'

animals = ['ariranha', 'capivara', 'jacaré-papo-amarelo', 'lobo-guará', 'onça-pintada', 'piranha', 'sucuri', 'tamanduá-bandeira',
           'tatu-canastra', 'tucano', 'pantanal']

!ls '/content/drive/My Drive/IMLDL/IMLDL-Desafio-DataSet'
!ls '/content/drive/My Drive/IMLDL/IMLDL-Desafio-Imagens'

ariranha.csv		 onça-pintada.csv  tamanduá-bandeira.csv
capivara.csv		 pantanal.csv	   tatu-canastra.csv
jacaré-papo-amarelo.csv  piranha.csv	   tucano.csv
lobo-guará.csv		 sucuri.csv
ariranha  jacaré-papo-amarelo  onça-pintada  piranha  tamanduá-bandeira  tucano
capivara  lobo-guará	       pantanal      sucuri   tatu-canastra


## Download das imagens
As células abaixo irão ler os arquivos csv e baixar as imagens para o jupyter notebook. Apenas rode se você não possuir as imagens no seu drive!

O número de imagens baixadas é definido pela variável *Num_pics*

In [ ]:
# Número de imagens a serem baixadas
Num_pics = 300

# Caminhos de destino para o .csv e pasta de cada animal
csv_paths = [fst.Path(dataset_path + '/' + animal + '.csv') for animal in animals] # fst.Path(dataset_path)
dest_paths = [fst.Path(destination_path + '/' + animal) for animal in animals] # fst.Path(dest_path)

In [ ]:
print(dest_paths)

/content/drive/My Drive/IMLDL/IMLDL-Desafio-Imagens/capivara


In [ ]:
files = {}
for i, animal in enumerate(animals):
  print('Reading: ', animal)
  fst.download_images(dest_paths[i], csv_paths[i], max_pics=Num_pics)
  for _,_,filenames in os.walk(dest_paths[i]):
    # print(filenames)
    files[animal] = [str(dest_paths[i]) + '/' + file for file in filenames]

Reading:  ariranha
Reading:  capivara
Reading:  jacaré-papo-amarelo
Reading:  lobo-guará
Reading:  onça-pintada
Reading:  piranha
Reading:  sucuri
Reading:  tamanduá-bandeira
Reading:  tatu-canastra
Reading:  tucano
Reading:  pantanal


In [ ]:
for animal in animals:
  print('Imagens para ', animal + ':', end=' ')
  dir = '/content/drive/My Drive/IMLDL/IMLDL-Desafio-Imagens/' + animal
  print(len(os.listdir(dir)))

Imagens para  ariranha: 277
Imagens para  capivara: 272
Imagens para  jacaré-papo-amarelo: 272
Imagens para  lobo-guará: 262
Imagens para  onça-pintada: 188
Imagens para  piranha: 236
Imagens para  sucuri: 239
Imagens para  tamanduá-bandeira: 250
Imagens para  tatu-canastra: 171
Imagens para  tucano: 215
Imagens para  pantanal: 190


## Criação do datset
Definimos as transformadas que serão aplicadas durante a criação do dataset


In [ ]:
# Processing data
img_shape = (224, 224)

transform = transforms.Compose([transforms.Resize(img_shape),
                                transforms.RandomHorizontalFlip(), transforms.RandomRotation(10),
                                transforms.ToTensor()])

In [ ]:
destination_path

'/content/drive/My Drive/IMLDL/IMLDL-Desafio-Imagens'

In [ ]:
# Utilzamos a função ImageFolder para criar o dataset ja com o label correto definido pela pasta
data = ImageFolder(destination_path, transform=transform)
data.class_to_idx

{'ariranha': 0,
 'capivara': 1,
 'jacaré-papo-amarelo': 2,
 'lobo-guará': 3,
 'onça-pintada': 4,
 'pantanal': 5,
 'piranha': 6,
 'sucuri': 7,
 'tamanduá-bandeira': 8,
 'tatu-canastra': 9,
 'tucano': 10}

In [ ]:
print('Total de imagens no dataset:', len(data))

Total de imagens no dataset: 2572


In [ ]:
percentage = 70

n_treino = round( len(data) * (percentage/100) )
n_teste = round( len(data) * (100 - percentage)/(2*100) )
n_valid = len(data) - n_treino - n_teste

print('nº de imagens para treino: {:}'.format(n_treino) +'; nº de imagens para teste: {:}'.format(n_teste) +'; nº de imagens para validação: {:}'.format(n_valid) + '.')
data_train, data_test, data_valid = random_split(data, [n_treino, n_teste, n_valid], generator=torch.Generator())

nº de imagens para treino: 1800; nº de imagens para teste: 386; nº de imagens para validação: 386.


In [ ]:
batch_size = 100

loader_train, loader_test, loader_valid = DataLoader(data_train, batch_size=batch_size), DataLoader(data_test, batch_size=1), DataLoader(data_valid, batch_size=1)

# Arquitetura da CNN

O time optou por utilizar a técnica de *Transfer Learning* e para isso utilizou uma rede pré-disponibilizada pelo Pytorch, a Resnet152. Algumas modificações são feitas com o intuito de adequar a rede ao problema de classificação de 11 classes, sendo 10 referetens às espécies e 1 referente ao bioma do pantanal.

In [ ]:
# Fazemos o download da rede pre-treinada
model = models.resnet152(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

# Modficamos a camada final para transformar as 2048 saídas da resnet em 11
model.fc = nn.Sequential(
    nn.Linear(2048, 1000),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(1000,11),
    nn.LogSoftmax(dim=1)
)

# Enviamos o modelo para a GPU
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


A seguir são defenidos alguns parâmetros do modelo, sendo o *Learning Rate*, a função de *Loss*, NLL, e o otimizador AdamW.

In [ ]:
lr = 0.0001

criterion = nn.NLLLoss()
optimizer = torch.optim.AdamW(model.fc.parameters(),lr)

A função definida abaixo tem o intuito de treinar a rede. Os argumentos da função são o modelo que foi instanciado nas células acima (*model*), o dataset de treino (*loader_train*) e também o dataset de test (*loader_test*). A função tem como retorno a *loss* do treinamento do modelo, a *loss* referente ao dataset de teste e a acurácia do modelo referente ao dataset de teste.

In [ ]:
def train_test(model, loader_train, loader_test):
  train_loss, test_loss, acc = 0,0,0

  # Modo de Treino
  model.train()
  training_loss = 0
  for img, label in loader_train:
    img, label = img.float().to(device), label.to(device)
    loss = criterion(model(img), label.float().long())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    training_loss += loss
  train_loss = training_loss/len(loader_train)

  # Modo de avaliação
  model.eval()
  correct = 0
  total = 0
  testing_loss = 0
  with torch.no_grad():
    for img, label in loader_test:
      img, label = img.float().to(device), label.to(device)
      out = model(img)
      _, predict = torch.max(out, dim=1)
      total += 1
      if predict == label:
        correct += 1
      testing_loss += criterion(out, label.long())
  test_loss = testing_loss/len(loader_test)
  acc = (correct/total)*100

  return train_loss, test_loss, acc

## Treinamento
A célula abaixo é responsável por realizar o treinamento do modelo. É definida a quantidade de épocas que o modelo será treinado e ao final de cada época são mostradas algumas informações, dentre elas a época, a loss da época, a acurácia e o tempo para a época. Aguarde 5 minutos durante a execução da célula pela primeira vez, caso não seja feito nenhum *print* reinicie a célula.

In [ ]:
START = time.time()
time1epoch = 0

# ========== Epocas ==========#
epochs = 15
# ============================

for epoch in range(1, epochs+1):
  start = time.time()
  train_loss, test_loss, acc = train_test(model, loader_train, loader_test)
  end = time.time()

  Time = end - start
  if epoch == 1:
    time1epoch = Time

  print('Epoch: ', epoch ,' loss: {:.4f}'.format(test_loss.item()), ' Accuracy: {:.2f}'.format(acc), ' Time spent this epoch: {:.2f}'.format(Time), 'seconds.')
END = time.time()

print()
TIME = (END - START) - time1epoch
print('\n Time spent during training, excluding first epoch: {:.2f}'.format(TIME), 'seconds.')	


Epoch:  1  loss: 0.2211  Accuracy: 98.19  Time spent this epoch: 21.24 seconds.
Epoch:  2  loss: 0.1884  Accuracy: 97.41  Time spent this epoch: 21.44 seconds.
Epoch:  3  loss: 0.1590  Accuracy: 97.15  Time spent this epoch: 20.84 seconds.
Epoch:  4  loss: 0.1288  Accuracy: 97.67  Time spent this epoch: 20.65 seconds.
Epoch:  5  loss: 0.1178  Accuracy: 98.70  Time spent this epoch: 20.86 seconds.
Epoch:  6  loss: 0.0996  Accuracy: 98.45  Time spent this epoch: 21.01 seconds.
Epoch:  7  loss: 0.0979  Accuracy: 98.45  Time spent this epoch: 20.94 seconds.
Epoch:  8  loss: 0.0987  Accuracy: 98.19  Time spent this epoch: 20.87 seconds.
Epoch:  9  loss: 0.0994  Accuracy: 97.41  Time spent this epoch: 20.96 seconds.
Epoch:  10  loss: 0.0863  Accuracy: 97.93  Time spent this epoch: 20.94 seconds.
Epoch:  11  loss: 0.0854  Accuracy: 97.93  Time spent this epoch: 20.89 seconds.
Epoch:  12  loss: 0.0783  Accuracy: 98.45  Time spent this epoch: 20.91 seconds.
Epoch:  13  loss: 0.0835  Accuracy: 9

## Validação
Com o intuito de obter uma outra forma de avaliar o modelo, é utilizado o dataset de validação para obter os valores de acurácia para imagens que não foram vistas pelo modelo. São dispostos os valores da acurácia para o dataset de validação, o erro médio e também o tempo gasto durante o treinamento, excluindo a primeira época.

In [ ]:
total = 0
correct = 0
loss_valid = 0

for imagem_valid, label_valid in loader_valid:

	loss_v = 0

	imagem_valid, label_valid = imagem_valid.float().to(device), label_valid.to(device)
	outputs_valid = model(imagem_valid)
	_, previsao = torch.max(outputs_valid, dim = 1)
	loss_v = criterion (outputs_valid, label_valid.long())
	total = total + 1 #Adiciona +1 na variável que guarda o total de previsões feitas
	if previsao == label_valid: 
		correct = correct + 1 #Soma +1 na variável que mede quantas previsões dessa categoria (erro absoluto de 2 graus) estão certas
	loss_valid += loss_v

loss_valid = loss_valid/len(loader_valid)
accuracy = (correct/total)*100 #Calcula a acurácia para erro absoluto de 1 grau em porcentagem

print('A acurácia obtida foi de: {:.2f}'.format(accuracy) + '%.')
print()
print('O erro médio obtido foi de: {:.4f}'.format(loss_valid))
print()
print('O tempo gasto durante o treinamento, excluindo a primeira época, foi de: {:.2f}'.format(TIME), 'segundos.')
print()

A acurácia obtida foi de: 97.67%.

O erro médio obtido foi de: 0.0694

O tempo gasto durante o treinamento, excluindo a primeira época, foi de: 606.50 segundos.



## Predict
A seguir será implementada a função *predict* responsável por retornar dois valores, o primeiro é um número que faz referência a qual das 11 classes foi identificada na imagem e o segundo valor pode assumir três valores distintos dependendo se a espécie em questão está em extinção, se a espécie não está em extinção ou até mesmo se na imagem não está presente nenhuma das espécies listadas. 

In [ ]:
def predict(image_tensor, model):
  '''
  0 - onça-pintada
  1 - lobo guará
  2 - ariranha
  3 - tatu canastra
  4 - tamanduá bandeira
  5 - jacaré do papo amarelo
  6 - sucuri
  7 - tucano
  8 - piranha
  9 - capivara
  10 - pantanal
  '''
  dic_aux = {0:2, 1:9, 2:5, 3:1, 4:0, 5:10, 6:8, 7:6, 8:4, 9:3, 10:7}
  '''
  O dicionário utilizado aqui tem como intuito adequar os índices de cada um dos
  animais com os que foram disponibilizados no roteiro do desafio final.
  Dessa forma os valores de data.class_to_idx são convertidos nos indíces seguindo
  o que foi proposto no roteiro.
  '''

  with torch.no_grad():
    out = model(image_tensor)
    _, predict = torch.max(out, dim=1)
    predict = predict.item()
    predict_animal = dic_aux[predict]
    if predict_animal < 5:
      conteudo_imagem = 'a'
    elif predict_animal < 10:
      conteudo_imagem = 'b'
    elif predict_animal == 10:
      conteudo_imagem = 'c'
    return predict_animal, conteudo_imagem




## Salvando o modelo
A célula abaixo salva o modelo no formato .pt na pasta IMLDL do google drive

In [ ]:
torch.save(model, '/content/drive/My Drive/IMLDL/model.pt')